In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
orders = pd.read_csv('orders.csv')
order_product_prior = pd.read_csv('order_products__prior.csv')
order_product_train = pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')
aisles=pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv') 
#train_order_df=pd.read_pickle('train_order_df.pkl')
products_feat=pd.read_pickle('product_feat_df.pkl')
users_feat=pd.read_pickle('user_feat_df.pkl')
user_product_feat=pd.read_pickle('user_product_feat.pkl')

merge orders and train orders

In [3]:
train_orders = orders.merge(order_product_train, on='order_id', how='inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [4]:
train_orders.drop(['order_id', 'eval_set','add_to_cart_order'],axis=1,inplace=True)
train_orders.head()

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered
0,1,11,4,8,14.0,196,1
1,1,11,4,8,14.0,25133,1
2,1,11,4,8,14.0,38928,1
3,1,11,4,8,14.0,26405,1
4,1,11,4,8,14.0,39657,1


In [5]:
train_orders.shape

(1384617, 7)

In [6]:
train_users = train_orders.user_id.unique()
train_users[:10]

array([ 1,  2,  5,  7,  8,  9, 10, 13, 14, 17], dtype=int64)

In [10]:
user_product_feat_df = user_product_feat[user_product_feat['user_id'].isin(train_users)]
user_product_feat.head()

,user_id,product_id,user_product_orders,user_product_reorders,user_product_reorder_ratio,user_product_add_to_cart_ratio,user_product_dow_ratio,user_product_hour_ratio,avg_days_since_last_purchase,reorder_in_third_last_order,reorder_in_second_last_order,reorder_in_last_order
0,1,196,10,9,0.900000,1.400000,2.500000,10.300000,17.600000,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,2.555556,10.555556,19.555556,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,4.000000,15.000000,28.000000,0.0,0.0,0.0
3,1,12427,10,9,0.900000,3.300000,2.500000,10.300000,17.600000,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,2.666667,8.000000,21.666667,1.0,0.0,0.0


In [11]:
user_product_feat_df.shape

(8474661, 12)

In [14]:
df = user_product_feat_df[user_product_feat_df['user_id'].isin(train_users)]

In [16]:
df=df.merge(train_orders, on=['user_id','product_id'], how='outer')
df.head()

,user_id,product_id,user_product_orders,user_product_reorders,user_product_reorder_ratio,user_product_add_to_cart_ratio,user_product_dow_ratio,user_product_hour_ratio,avg_days_since_last_purchase,reorder_in_third_last_order,reorder_in_second_last_order,reorder_in_last_order,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.900000,1.400000,2.500000,10.300000,17.600000,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888889,3.333333,2.555556,10.555556,19.555556,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,0.0,0.000000,5.000000,4.000000,15.000000,28.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1,12427,10.0,9.0,0.900000,3.300000,2.500000,10.300000,17.600000,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1,13032,3.0,2.0,0.666667,6.333333,2.666667,8.000000,21.666667,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0


In [17]:
df.shape

(9030454, 17)

In [18]:
df.order_number.fillna(df.groupby('user_id')['order_number'].transform('mean'), inplace = True)
df.order_dow.fillna(df.groupby('user_id')['order_dow'].transform('mean'), inplace = True)
df.order_hour_of_day.fillna(df.groupby('user_id')['order_hour_of_day'].transform('mean'), inplace = True)
df.days_since_prior_order.fillna(df.groupby('user_id')['days_since_prior_order'].\
                                                             transform('mean'), inplace = True)

In [19]:
df.reordered.value_counts()

1.0    828824
0.0    555793
Name: reordered, dtype: int64

In [23]:
df['reordered'].isnull().sum()

7645837

In [25]:
df = df[df['reordered']!=0]

In [27]:
df.shape

(8474661, 17)

In [ ]:
train_order_df = train_order_df.merge(users_feat, on='user_id', how='left')
train_order_df = train_order_df.merge(products_feat, on='product_id', how='left')
train_order_df.head()

In [28]:
df['reordered'].fillna(0, inplace=True)
df.isnull().sum()

C:\Users\MUH0351\AppData\Local\Temp\ipykernel_2992\2675908918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reordered'].fillna(0, inplace=True)


user_id                           0
product_id                        0
user_product_orders               0
user_product_reorders             0
user_product_reorder_ratio        0
user_product_add_to_cart_ratio    0
user_product_dow_ratio            0
user_product_hour_ratio           0
avg_days_since_last_purchase      0
reorder_in_third_last_order       0
reorder_in_second_last_order      0
reorder_in_last_order             0
order_number                      0
order_dow                         0
order_hour_of_day                 0
days_since_prior_order            0
reordered                         0
dtype: int64

In [33]:
df.head()

,user_id,product_id,user_product_orders,user_product_reorders,user_product_reorder_ratio,user_product_add_to_cart_ratio,user_product_dow_ratio,user_product_hour_ratio,avg_days_since_last_purchase,reorder_in_third_last_order,reorder_in_second_last_order,reorder_in_last_order,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.900000,1.400000,2.500000,10.300000,17.600000,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888889,3.333333,2.555556,10.555556,19.555556,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,0.0,0.000000,5.000000,4.000000,15.000000,28.000000,0.0,0.0,0.0,11.0,4.0,8.0,14.0,0.0
3,1,12427,10.0,9.0,0.900000,3.300000,2.500000,10.300000,17.600000,1.0,1.0,1.0,11.0,4.0,8.0,14.0,0.0
4,1,13032,3.0,2.0,0.666667,6.333333,2.666667,8.000000,21.666667,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0


In [30]:
df.to_pickle('train_df.pkl')